In [9]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])

#Display first 3 rows of scaled data
df_market_data_scaled[0:3]

In [ ]:
## Create a DataFrame with the scaled data
# Copy the crypto names from the original DataFrame

df_market_data_scaled = pd.DataFrame(
    df_market_data_scaled,
    columns=["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"],
    index=df_market_data.index
)

# Display the scaled DataFrame
df_market_data_scaled.head(10)

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia_scaled = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia_scaled.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_lower = model.predict(df_market_data_scaled)


# Print the resulting array of cluster values.
print("Cluster values:", k_lower)

In [ ]:
# Create a copy of the scaled DataFrame
df_market_data_scaled_predictions = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_scaled_predictions['clusters_lower'] = k_lower

# Display the copy of the scaled DataFrame
df_market_data_scaled_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters_lower",
    hover_cols=["coin_id"],
    height=600,
    width=800,
).opts(yformatter="%.0f")

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled)


# View the scaled PCA data
market_data_pca[:5]

In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Total explained variance = 0.89503166 or approximately 89.50%

In [ ]:
# Create a new DataFrame with the PCA data.
# Copy the crypto names from the original scaled DataFrame
# Set the coin_id column as index
df_market_data_pca = pd.DataFrame(
    market_data_pca,
    columns=["PCA1", "PCA2", "PCA3"],
    index=df_market_data_scaled.index
)

# Display the scaled PCA DataFram
df_market_data_pca.head()

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_pca)
    inertia_pca.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k":k, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
k_4 = model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print("Cluster values:", k_4)

In [ ]:
# Create a copy of the scaled PCA DataFrame
df_market_data_pca_predictions = df_market_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_market_data_pca_predictions["cluster_segments"] = k_4

# Display the copy of the scaled PCA DataFrame
df_market_data_pca_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_pca_predictions.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="cluster_segments",
    hover_cols=["coin_id"],
    height=600,
    width=800,
).opts(yformatter="%.0f")

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves

# Create DF to hold k-values and inertia for both datasets
elbow_data = pd.DataFrame({
    'k': list(range(1, 12)),
    'Scaled Data': inertia_scaled,
    'PCA Data': inertia_pca
})

# Create composite line plot
elbow_plot = elbow_data.hvplot.line(
    x='k',
    y=['Scaled Data', 'PCA Data'],
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
    line_width=2,
    legend='top_right',
    height=500,
    width=800
)

# Display plot
elbow_plot

In [ ]:
# Composite plot to contrast the clusters

# Reference the first scatter plot for scaled data
scatter_scaled = df_market_data_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters_lower",
    hover_cols=["coin_id"],
    height=600,
    width=400,
).opts(yformatter="%.0f", title="Clusters from Scaled Data")

# Reference the second scatter plot for PCA data
scatter_pca = df_market_data_pca_predictions.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="cluster_segments",
    hover_cols=["coin_id"],
    height=600,
    width=400,
).opts(yformatter="%.0f", title="Clusters from PCA Data")

# Combine the two plots side by side
composite_plot = scatter_scaled + scatter_pca

# Display the composite plot
composite_plot.opts(title="Comparison of Scaled Data and PCA Clusters")

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** It appears that the customer segmentation information using the DataFrame with all of the factors yielded similar results that the PCA analysis. The data is grouped in clear cluster segments in both cases.